In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from pathlib import Path

import hydra
import torch
import wandb
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

from opr.datasets.dataloader_factory import make_dataloaders
from opr.testing import test
from opr.training import epoch_loop

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
from hydra import initialize, compose
from hydra.utils import instantiate

with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name='config_text.yaml')  # only_text
    # cfg = compose(config_name='config_default.yaml')  # lidar + cam

In [4]:
if not cfg.general.debug and not cfg.wandb.disabled:
    config_dict = OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True)
    wandb.init(
        name=cfg.wandb.run_name,
        project=cfg.wandb.project,
        settings=wandb.Settings(start_method="thread"),
        config=config_dict,
    )
    wandb.save(f"configs/{wandb.run.name}.yaml")
    run_name = wandb.run.name
else:
    run_name = "debug"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: petili. Use `wandb login --relogin` to force relogin


In [5]:
checkpoints_dir = (
    Path(cfg.general.checkpoints_dir) / f"{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}_{run_name}"
)
if not checkpoints_dir.exists():
    checkpoints_dir.mkdir(parents=True)

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump

train_csv_path = "/home/docker_opr/Datasets/splitted/train/train.csv"

df = pd.read_csv(train_csv_path)

def train_tfidf_pca(corpus, n_components=100, base_savepath="./opr/datasets/"):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(corpus)
    vectorized_corpus = vectorizer.transform(corpus).toarray()

    pca =PCA(n_components=n_components)
    pca.fit(vectorized_corpus)
    
    vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
    pca_savepath = os.path.join(base_savepath, 'pca.joblib')
    dump(vectorizer, vectorizer_savepath) 
    dump(pca, pca_savepath) 

def text_transform(self, text):
    vect_data = self.vectorizer.transform([text]).toarray()
    pca_data = self.pca.transform(vect_data)
    pca_data = torch.tensor(pca_data, dtype=torch.float32)
    return pca_data

In [7]:
if "text" in cfg.general.modalities:
    corpus = np.hstack((df["back_description"], df["front_description"]))
    train_tfidf_pca(corpus)

In [8]:
print("=> Instantiating model...")
model = instantiate(cfg.model)

print("=> Instantiating loss...")
loss_fn = instantiate(cfg.loss)

print("=> Making dataloaders...")
dataloaders = make_dataloaders(
    dataset_cfg=cfg.dataset.dataset,
    batch_sampler_cfg=cfg.dataset.sampler,
    num_workers=cfg.dataset.num_workers,
)

=> Instantiating model...
=> Instantiating loss...
=> Making dataloaders...


In [9]:
cfg

{'general': {'debug': False, 'seed': 31299, 'checkpoints_dir': 'checkpoints/', 'device': 'cuda', 'num_workers': 4, 'batch_expansion_th': 0.7, 'modalities': ['text'], 'epochs': 40}, 'loss': {'_target_': 'opr.losses.MultimodalTripletMarginLoss', 'margin': 0.2, 'swap': True, 'distance': {'_target_': 'pytorch_metric_learning.distances.LpDistance', 'normalize_embeddings': False, 'collect_stats': True}, 'miner': {'_target_': 'opr.miners.HardTripletMiner', 'distance': {'_target_': 'pytorch_metric_learning.distances.LpDistance', 'normalize_embeddings': False}, 'modalities': '${loss.modalities}'}, 'reducer': {'_target_': 'pytorch_metric_learning.reducers.AvgNonZeroReducer', 'collect_stats': True}, 'modalities': ['text'], 'weights': [1]}, 'wandb': {'disabled': False, 'project': 'OpenPlaceRecognition', 'run_name': 'only_text'}, 'model': {'_target_': 'opr.models.base_models.ComposedModel', 'image_module': None, 'cloud_module': None, 'fusion_module': None, 'text_module': {'_target_': 'opr.models.ba

In [10]:
print("=> Instantiating optimizer...")
params_list = []
for modality in cfg.general.modalities:
    params_list.append(
        {
            "params": getattr(model, f"{modality}_module").parameters(),
            "lr": cfg.optimizer.learning_rates[f"{modality}_lr"],
        }
    )
optimizer = instantiate(cfg.optimizer.fn, params=params_list)
print("Instantiating scheduler...")
scheduler = instantiate(cfg.scheduler, optimizer=optimizer)

model = model.to(cfg.general.device)

=> Instantiating optimizer...
Instantiating scheduler...


In [15]:
epoch

79

In [17]:
# ==========> TRAIN LOOP:

# best_recall_at_1 = 0.0

# for epoch in range(cfg.general.epochs):
for epoch in range(120, 180):
    print(f"\n\n=====> Epoch {epoch+1}:")
    # TODO: resolve mypy typing here
    train_batch_size = dataloaders["train"].batch_sampler.batch_size  # type: ignore

    print("\n=> Training:\n")

    train_stats, train_rate_non_zero = epoch_loop(
        dataloader=dataloaders["train"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        phase="train",
        device=cfg.general.device,
    )

    print(f"\ntrain_rate_non_zero = {train_rate_non_zero}")

    batch_expansion_th = cfg.general.batch_expansion_th
    if batch_expansion_th is not None:
        if batch_expansion_th == 1.0:
            print("Batch expansion rate is set to every epoch. Increasing batch size.")
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore
        elif train_rate_non_zero is None:
            print(
                "\nWARNING: 'batch_expansion_th' was set, but 'train_rate_non_zero' is None. ",
                "The batch size was not expanded.",
            )
        elif train_rate_non_zero < batch_expansion_th:
            print(
                "Average non-zero triplet ratio is less than threshold: ",
                f"{train_rate_non_zero} < {batch_expansion_th}",
            )
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore

    print("\n=> Validating:\n")

    val_stats, val_rate_non_zero = epoch_loop(
        dataloader=dataloaders["val"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        phase="val",
        device=cfg.general.device,
    )

    print(f"\nval_rate_non_zero = {val_rate_non_zero}")

    print("\n=> Testing:\n")

    recall_at_n, recall_at_one_percent, mean_top1_distance = test(
        model=model,
        descriptor_key="text",
        dataloader=dataloaders["test"],
        device=cfg.general.device,
    )

    stats_dict = {}
    stats_dict["test"] = {
        "mean_top1_distance": mean_top1_distance,
        "recall_at_1%": recall_at_one_percent,
        "recall_at_1": recall_at_n[0],
        "recall_at_3": recall_at_n[2],
        "recall_at_5": recall_at_n[4],
        "recall_at_10": recall_at_n[9],
    }
    stats_dict["train"] = train_stats
    stats_dict["train"]["batch_size"] = train_batch_size
    stats_dict["val"] = val_stats

    # saving checkpoints
    checkpoint_dict = {
        "epoch": epoch + 1,
        "stats_dict": stats_dict,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }
    torch.save(checkpoint_dict, checkpoints_dir / f"epoch_{epoch+1}.pth")
    if recall_at_n[0] > best_recall_at_1:
        print("Recall@1 improved!")
        torch.save(checkpoint_dict, checkpoints_dir / "best.pth")
        best_recall_at_1 = recall_at_n[0]
        
    wandb.log({'Test recall_at_1': recall_at_n[0],
                'Test Mean Recall@1%': recall_at_one_percent,
                    'Test Mean top-1 distance': mean_top1_distance,
                    'batch_size': train_stats['batch_size'],
                    'train_total_loss': train_stats['total_loss'],
                    'val_total_loss': val_stats['total_loss']})



=====> Epoch 121:

=> Training:



train: 100%|██████████| 91/91 [00:05<00:00, 17.91it/s]


train_rate_non_zero = 0.30405093990258825
Average non-zero triplet ratio is less than threshold:  0.30405093990258825 < 0.7

=> Validating:




val: 100%|██████████| 34/34 [00:01<00:00, 24.53it/s]



val_rate_non_zero = 1.0

=> Testing:



...


train_rate_non_zero = 0.16481227106227106
Average non-zero triplet ratio is less than threshold:  0.16481227106227106 < 0.7

=> Validating:




val: 100%|██████████| 33/33 [00:01<00:00, 18.45it/s]



val_rate_non_zero = 0.9900072150072151

=> Testing:



Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 17.68it/s]

Mean Recall@N:
[0.69636809 0.74398884 0.76537798 0.78081354 0.79352898 0.80597559
 0.81652901 0.82871759 0.83519602 0.8425223  0.85064778 0.85768223
 0.86419872 0.87069927 0.87612108 0.87963827 0.8855865  0.88910315
 0.89450408 0.89775329 0.90207961 0.90777151 0.91156432 0.91454786
 0.91779315]
Mean Recall@1% = 0.8059755947133013
Mean top-1 distance = 0.2686481071426822


In [12]:
torch.concat(list((data["image"], data["cloud"], data["text"])), dim=1)

NameError: name 'data' is not defined

In [ ]:
list(data["image"], data["cloud"])

In [ ]:
list((1, 2))

[1, 2]

In [ ]:
c = {"a" :1, 
     "b" : 2}